## 0. Carregamento de Bibliotecas e Dados

In [ ]:
import tqdm
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import seaborn as sns
import re
from difflib import get_close_matches
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

In [ ]:
df_cad = pd.read_csv("./processed_data/cadastro.csv")
df_docs = pd.read_csv("./processed_data/medicoes.csv")

In [ ]:
df_docs.head()

## 1. Coluna **Item medição**

Essa coluna é responsavel por dizer qual a categoria do item a ser medido. Apesar de estar em formato de texto, não há informaçẽos extras a serem extraidas daqui.

In [4]:
df_docs['Item medição'].unique()

array(['VÃO', 'CABO CONDUTOR', 'CABO PARA RAIO', 'ESTRUTURA',
       'FERRAGENS DE CONDUTO', 'LOCALIZAÇÃO DA TORRE', 'ISOLADOR',
       'FERRAGENS DE CABO GU', 'SISTAMA DE ATERRAMEN'], dtype=object)

## 2. Coluna **Txt.code codif.**

A coluna "Txt.code codif." contém códigos de categorização dos problemas encontrados nas inspeções. Abaixo, será criado um mapeamento para agrupar esses códigos em categorias mais amplas e organizadas. Este agrupamento facilitará a análise e visualização dos dados, reduzindo a dimensionalidade e permitindo identificar padrões nos tipos de problemas mais comuns.

Alem disso, criamos uma coluna que classifica o sentimento da categoria (positivo, negativo e neutro.)

Esse agrupamento foi gerado com o auxilio de LLM's (como o ChatGPT)


In [9]:
category_to_group = {
    # 1. Status Geral
    'CONDIÇÃO NORMAL': 'Status Geral',
    'CONDIÇÃO PERIGOSA': 'Status Geral',
    'Desconhecido':        'Status Geral',
    'INTERROMPIDO':        'Status Geral',
    'INEXISTENTE':         'Status Geral',
    'ROÇAR OU REABRIR':    'Status Geral',
    'RISCO FUTURO':        'Status Geral',

    # 2. Localização
    'LADO DIREITO':      'Localização',
    'LADO ESQUERDO':     'Localização',
    'AMBOS OS LADOS':    'Localização',

    # 3. Componentes faltantes
    'ATERRAMENTO FALTANTE':           'Faltantes',
    'SECCIONAMENTO FALTANTE':         'Faltantes',
    'ATERR. E SECC. FALTANTE':        'Faltantes',
    'FALTANDO':                       'Faltantes',
    'FALTANDO BALISAMENTO':           'Faltantes',
    'CUPILHA/CONTRAPINO FALTANTE':    'Faltantes',
    'IDENTIFICAÇÃO FALTANTE':         'Faltantes',
    'ADVERTÊNCIA FALTANTE':           'Faltantes',
    'PROTETOR ANTI-PASSARO FALTANTE': 'Faltantes',
    'PARAFUSO/PORCA FALTANDO/FROUXO':'Faltantes',

    # 4. Instalação / Conformidade
    'MAL INSTALADO':     'Instalação',
    'INADEQUADO':        'Instalação',
    'FORA DE PRUMO':     'Instalação',
    'CUPILHA/CONTRAPINO MAL INSTALADA': 'Instalação',
    'FERRAGEM':          'Instalação',

    # 5. Corrosão e Desgaste
    'CORROSÃO GRAU I':        'Corrosão',
    'CORROSÃO GRAU II':       'Corrosão',
    'CORROSÃO GRAU III':      'Corrosão',
    'COM DESGASTE INICIAL':   'Desgaste',
    'COM DESGASTE MÉDIO':     'Desgaste',
    'COM DESGASTE ACENTUADO': 'Desgaste',

    # 6. Danos e Exposição
    'DANIFICADO':        'Danos',
    'ROMPIDO':           'Danos',
    'CHAMUSCADO':        'Danos',
    'QUEIMADA NA FAIXA': 'Danos',
    'EXPOSTO':           'Danos',

    # 7. Objetos Estranhos e Animais
    'COM OBJETO ESTRANHO':           'Objetos Estranhos',
    'COM OBJETO ESTRANHO EM RISCO':  'Objetos Estranhos',
    'PIPA/RABIOLA':                  'Objetos Estranhos',
    'NINHO NO CORPO DA ESTRUTURA':   'Objetos Estranhos',
    'NINHO EM LOCAL DE RISCO':       'Objetos Estranhos',

    # 8. Deformações e Tensão
    'CABO ENGAIOLADO':          'Deformação',
    'ENGAIOLADO':               'Deformação',
    'ESFORÇO UNILATERAL NA MÍSULA':'Deformação',

    # 9. Conexões e Emendas
    'CONEXAO':                         'Conexões/Emendas',
    'EMENDA':                          'Conexões/Emendas',
    'BOBINA DE BLOQUEIO':              'Conexões/Emendas',
    'CONEXÃO APARAFUSADA CC':          'Conexões/Emendas',
    'CONEXÕES PRENSADAS CC-CHAPA/CHAPA':'Conexões/Emendas',
    'EMENDAS PRÉ-FORMADA CC':          'Conexões/Emendas',
    'EMENDAS PRENSADAS CC':            'Conexões/Emendas',
    'CONEXÕES PRENSADAS CC/CONECTOR':  'Conexões/Emendas',
    'EMENDA PRÉ-FORMADA E PRENSADA':   'Conexões/Emendas',

    # 10. Identificação e Avisos
    'IDENTIFICAÇÃO APAGADA': 'Identificação/Avisos',
    'ADVERTÊNCIA APAGADA':   'Identificação/Avisos',

    # 11. Sinalização Visual
    'SEM VISUALIZAÇÃO (DRONE)': 'Inspeção',

    # 12. Vegetação e Terreno
    'ROÇADA MOTORIZADA':     'Vegetação',
    'ALTERAÇÃO DO TERRENO':  'Terreno',

    # 13. Proximidade de Componentes
    'AFASTADO DE COMPONENTES':  'Proximidade',
    'PROXIMO DE COMPONENTES':   'Proximidade',

    # 14. Obstáculos e Invasões
    'MURO':                         'Obstáculos/Invasões',
    'FS - MURETA':                  'Obstáculos/Invasões',
    'FS - CALÇADA':                 'Obstáculos/Invasões',
    'FS - INVASÃO - COM NOTIFICACAO':'Obstáculos/Invasões',
    'FS - MURO - COM NOTIFICACAO':  'Obstáculos/Invasões',
    'FS - MURETA - COM NOTIFICACAO':'Obstáculos/Invasões',
    'FS - CALÇADA - COM NOTIFICACAO':'Obstáculos/Invasões',

    # 15. Posição de Vão
    'INICIO DO VAO':         'Vão',
    'MEIO DO VAO':           'Vão',
    'FIM DO VAO':            'Vão',
    'INICIO E MEIO DO VAO':  'Vão',
    'MEIO E FIM DO VAO':     'Vão',
    'INICIO E FIM DO VAO':   'Vão',
    'VAO COMPLETO':          'Vão',

    # 16. Outros
    'SIM':                   'Outros',
    'SUJO / POLUIDO':        'Outros',
    'DISCOS QUEBRADOS - CADEIA LEVE': 'Outros',
    'DISCOS QUEBRADOS - CADEIA PESADA':'Outros'
}

df_docs['resposta_categoria'] = df_docs['Txt.code codif.'].apply(lambda x: category_to_group.get(x, 'Outros'))


In [11]:
classification = {
    # 1. Status Geral
    'CONDIÇÃO NORMAL':       'positivo',
    'CONDIÇÃO PERIGOSA':     'negativo',
    'Desconhecido':          'neutro',
    'INTERROMPIDO':          'negativo',
    'INEXISTENTE':           'negativo',
    'ROÇAR OU REABRIR':      'neutro',
    'RISCO FUTURO':          'negativo',

    # 2. Localização
    'LADO DIREITO':          'neutro',
    'LADO ESQUERDO':         'neutro',
    'AMBOS OS LADOS':        'neutro',

    # 3. Componentes faltantes
    'ATERRAMENTO FALTANTE':            'negativo',
    'SECCIONAMENTO FALTANTE':          'negativo',
    'ATERR. E SECC. FALTANTE':         'negativo',
    'FALTANDO':                        'negativo',
    'FALTANDO BALISAMENTO':            'negativo',
    'CUPILHA/CONTRAPINO FALTANTE':     'negativo',
    'IDENTIFICAÇÃO FALTANTE':          'negativo',
    'ADVERTÊNCIA FALTANTE':            'negativo',
    'PROTETOR ANTI-PASSARO FALTANTE':  'negativo',
    'PARAFUSO/PORCA FALTANDO/FROUXO':  'negativo',

    # 4. Instalação / Conformidade
    'MAL INSTALADO':        'negativo',
    'INADEQUADO':           'negativo',
    'FORA DE PRUMO':        'negativo',
    'CUPILHA/CONTRAPINO MAL INSTALADA': 'negativo',
    'FERRAGEM':             'neutro',

    # 5. Corrosão e Desgaste
    'CORROSÃO GRAU I':        'negativo',
    'CORROSÃO GRAU II':       'negativo',
    'CORROSÃO GRAU III':      'negativo',
    'COM DESGASTE INICIAL':   'negativo',
    'COM DESGASTE MÉDIO':     'negativo',
    'COM DESGASTE ACENTUADO': 'negativo',

    # 6. Danos e Exposição
    'DANIFICADO':        'negativo',
    'ROMPIDO':           'negativo',
    'CHAMUSCADO':        'negativo',
    'QUEIMADA NA FAIXA': 'negativo',
    'EXPOSTO':           'negativo',

    # 7. Objetos Estranhos e Animais
    'COM OBJETO ESTRANHO':          'negativo',
    'COM OBJETO ESTRANHO EM RISCO': 'negativo',
    'PIPA/RABIOLA':                 'negativo',
    'NINHO NO CORPO DA ESTRUTURA':  'negativo',
    'NINHO EM LOCAL DE RISCO':      'negativo',

    # 8. Deformações e Tensão
    'CABO ENGAIOLADO':             'negativo',
    'ENGAIOLADO':                  'negativo',
    'ESFORÇO UNILATERAL NA MÍSULA':'negativo',

    # 9. Conexões e Emendas
    'CONEXAO':                         'neutro',
    'EMENDA':                          'neutro',
    'BOBINA DE BLOQUEIO':              'neutro',
    'CONEXÃO APARAFUSADA CC':          'neutro',
    'CONEXÕES PRENSADAS CC-CHAPA/CHAPA':'neutro',
    'EMENDAS PRÉ-FORMADA CC':          'neutro',
    'EMENDAS PRENSADAS CC':            'neutro',
    'CONEXÕES PRENSADAS CC/CONECTOR':  'neutro',
    'EMENDA PRÉ-FORMADA E PRENSADA':   'neutro',

    # 10. Identificação e Avisos
    'IDENTIFICAÇÃO APAGADA': 'negativo',
    'ADVERTÊNCIA APAGADA':   'negativo',

    # 11. Sinalização Visual
    'SEM VISUALIZAÇÃO (DRONE)': 'neutro',

    # 12. Vegetação e Terreno
    'ROÇADA MOTORIZADA':     'negativo',
    'ALTERAÇÃO DO TERRENO':  'neutro',

    # 13. Proximidade de Componentes
    'AFASTADO DE COMPONENTES': 'neutro',
    'PROXIMO DE COMPONENTES':  'neutro',

    # 14. Obstáculos e Invasões
    'MURO':                         'negativo',
    'FS - MURETA':                  'negativo',
    'FS - CALÇADA':                 'negativo',
    'FS - INVASÃO - COM NOTIFICACAO':'negativo',
    'FS - MURO - COM NOTIFICACAO':  'negativo',
    'FS - MURETA - COM NOTIFICACAO':'negativo',
    'FS - CALÇADA - COM NOTIFICACAO':'negativo',

    # 15. Posição de Vão
    'INICIO DO VAO':         'neutro',
    'MEIO DO VAO':           'neutro',
    'FIM DO VAO':            'neutro',
    'INICIO E MEIO DO VAO':  'neutro',
    'MEIO E FIM DO VAO':     'neutro',
    'INICIO E FIM DO VAO':   'neutro',
    'VAO COMPLETO':          'neutro',

    # 16. Outros
    'SIM':                           'neutro',
    'SUJO / POLUIDO':                'negativo',
    'DISCOS QUEBRADOS - CADEIA LEVE':  'negativo',
    'DISCOS QUEBRADOS - CADEIA PESADA':'negativo',
}
df_docs['resposta_categoria_sentimento'] = df_docs['Txt.code codif.'].apply(lambda x: classification.get(x, 'Outros'))


In [ ]:
a = df_docs[df_docs['resposta_categoria_sentimento'] != 'Outros']
contagem = a.groupby('resposta_categoria_sentimento').size().sort_values(ascending=False)

plt.figure(figsize=(12,6))

sns.barplot(x=contagem.index, y=contagem.values)

# Customizar o gráfico
plt.title('Contagem por Categoria', fontsize=16)
plt.xlabel('Categoria', fontsize=14)
plt.ylabel('Quantidade', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.tight_layout()

plt.show()

In [11]:
del df_docs['Txt.code codif.']
del df_docs['Cód.valorização']

## 3. Coluna **Denominação**

Abaixo, agrupamos as categorias de denominação existente em super-categorias.

In [12]:


CATEGORY_SUBCATEGORY_MAP = {
    'Vegetação e Limpeza': [
        'ROCADA MECANIZADA', 'ROCADA MANUAL', 'PODA', 'ENTULHO', 'INVASAO',
        'Ninho De Pássaro'
    ],
    'Corrosão': [
        r'CABO PARA-RAIO.*', r'CABO CONDUTOR.*', r'FERRAGENS/CONECTORES.*',
        r'CORROSAO.*', r'EMENDA/REPARO.*', r'JUMPER.*', r'ESPACADOR.*',
        r'AMORTECEDOR.*', r'ESFERA.*', r'PROTECAO.*', r'.*STUB', 
        r'CORROS.*', r'CORRO.*',  r'COR.*', 
    ],
    'Isoladores': [
        r'ISOL.*', r'JUMPERS?-.*'
    ],
    'Temperatura': [
        r'TEMP.*'
    ],
    'Distâncias': [
        r'DISTANCIA.*', r'DISTAN.*'
    ],
    'Base e Fundação': [
        r'BASE.*', r'FUNDACAO.*', r'STUB'
    ],
    'Grampos e Fixadores': [
        r'GRAMPO.*', r'PARAFUSO.*', r'PARAFUSOS.*'
    ],
    'Emendas e Jumpers': [
        r'EMENDA.*', r'REPARO.*', r'JUMPER.*', r'BOBINA DE BLOQUEIO.*'
    ],
    'Amortecedores': [
        r'AMORTECEDOR.*'
    ],
    'Proteções e Acessórios': [
        r'PROTECAO.*', r'CONECTOR.*', r'CABO/FITA DE CONTRAPESO.*',
        r'PROTEÇÃO.*',r'PROTECÃO.*', r'PROTEÇAO.*',
    ],
    'Sinalização Aérea': [
        r'ESFERA.*', r'SINALIZACAO AEREA'
    ],
    'Componentes Estruturais': [
        r'ESTRUTURA.*', r'PECA.*', r'VAO', r'PEDAROL'
    ],
    'Tentos Rompidos': [
        r'TENTO ROMP.*'
    ],
    'Acesso e Segurança': [
        r'CONDICAO DO ACESSO', r'TRAVESSIA IRREGULAR', r'SECCINAMENTO.*',
        r'ESTREITAMENTO.*'
    ],
    'Desgastes': [
        r'DESGA.*'
    ],
}


def get_category(label: str) -> str:
    for category, patterns in CATEGORY_SUBCATEGORY_MAP.items():
        for pat in patterns:
            if re.match(pat.lower(), label.lower()):
                return category
    return 'Outros'


def get_subcategory(label: str) -> str:
    return label.strip().title()

In [14]:
df_docs['Denominação_categoria'] = df_docs['Denominação'].apply(get_category)
df_docs['Denominação_subcategoria'] = df_docs['Denominação'].apply(get_subcategory)


del df_docs['Denominação']

In [15]:
df_docs.head()

,Equipamento,Ponto medição,Doc.medição,Data,Item medição,Valor teórico,LimInfIntMed.,LimSupIntMed.,ValMed/PosTCont,Unid.caracter.,...,inside_limits,desvio,condicao_txt,condicao_cod,resposta_txt,resposta_cod,resposta_categoria,resposta_categoria_sentimento,Denominação_categoria,Denominação_subcategoria
0,323958,10155659,7991396,2025-02-18,VÃO,0.0,0.0,100.0,1500.0,m2,...,False,14.0,não especificado,0,fim do vao,66,Vão,neutro,Vegetação e Limpeza,Rocada Mecanizada
1,323957,10155490,7991397,2025-02-18,VÃO,0.0,0.0,100.0,5400.0,m2,...,False,53.0,não especificado,0,inicio do vao,64,Vão,neutro,Vegetação e Limpeza,Rocada Manual
2,323957,10155491,7991398,2025-02-18,VÃO,0.0,0.0,100.0,3600.0,m2,...,False,35.0,não especificado,0,meio e fim do vao,68,Vão,neutro,Vegetação e Limpeza,Rocada Mecanizada
3,323955,10155322,7991399,2025-02-18,VÃO,0.0,0.0,100.0,6000.0,m2,...,False,59.0,não especificado,0,meio e fim do vao,68,Vão,neutro,Vegetação e Limpeza,Rocada Manual
4,323954,10155155,7991400,2025-02-18,VÃO,0.0,0.0,100.0,10350.0,m2,...,False,102.5,não especificado,0,vao completo,70,Vão,neutro,Vegetação e Limpeza,Rocada Mecanizada
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247823,101272,4184755,3344707,2021-08-23,CABO CONDUTOR,75.0,0.0,999.0,0.0,°C,...,True,0.0,condição normal,1,desconhecido,-1,Status Geral,positivo,Temperatura,Temp-Inf/Dir/Exter(Circ Dir/Uni)
247824,101272,4184755,3200735,2021-07-02,CABO CONDUTOR,75.0,0.0,999.0,126.0,°C,...,True,0.0,não especificado,0,conexao,41,Conexões/Emendas,neutro,Temperatura,Temp-Inf/Dir/Exter(Circ Dir/Uni)
247825,101272,6506812,5970990,2023-11-09,FERRAGENS DE CONDUTO,0.0,0.0,6.0,0.0,UN,...,True,0.0,condição normal,1,desconhecido,-1,Status Geral,positivo,Corrosão,Ferragens/Conectores Condutor (Esq)
247826,101272,6506812,5970837,2023-11-09,FERRAGENS DE CONDUTO,0.0,0.0,6.0,0.0,UN,...,True,0.0,condição normal,1,desconhecido,-1,Status Geral,positivo,Corrosão,Ferragens/Conectores Condutor (Esq)


## 4. Coluna **Texto**

A coluna texto contem informações não estruturadas, inseridas pelos operadores, portanto, precisamos usar um modelo de LLM para classificar o sentimento do texto informado pelos operadores.

In [16]:
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)



In [17]:
llm_map = {}

In [18]:
import tqdm

texts = df_docs['Texto'].unique()


# 3. Lista de keywords negativas (domínio linhas de transmissão)

NEGATIVO = [
    "roçada", "rocada", "roçar", "poda", "ninho", "anti pássaro", "contra pássaros",
    "cortar", "cipó", "sobreaquecimento", "moderada", "vegetação", "bambu",
    "colonião", "leira de", "árvore", "arvinhas", "coqueiro", "capim",
    "cerca viva", "pipas", "eucalipto", "base suja", "danificado",
    "corrosão", "corros", "corosão", "severa", "erosão",
    "curicaca", "peças torcidas", "limp", "formigueiro",
    "rabiolas", "romp", "furtadas", "substituir",
    "quebrad",  "quebrado", "quebrada",
    "solto", "corrigir", "limpar", "quebrado", "buraco", "ninho", "corrosão", "cortar", "roçada", "base suja",
    "desconectado", "solto", "ferrugem", "isolador", "vazamento", "trincado",
    "danificado", "queda", "inclinado", "toco", "galho", "pendurado",
    "obstrução", "vegetação", 'desgaste', 'rompido', 'suja', 'escrementos', 'faltante', 'entulho', 'impossível', 'peça torta',
    'ausência', 'bambuzinho', 'cerca viva', 'vegetação', 'arvores', 'Árvores', 'Veget', 'vegetac', 'vegetaç', 'Faltam ',
    'Falta', 'cupilha', 'esferas descoloridas', 'dejetos aves', 'dejetos pássaro', 'esfera sinalização', 'peças soltas', 'terra exposto',
    'conector jumper inadequado', 'pino concha afastado', 'cupilha mal instalada', 'furto peças base', 'cavalote torto',
    'pino manilha sem porca',
    'TDA jumper mal instalado', 'ramal cruzaço aquecimento', 'aquecimento C2', 'notificação prefeitura', 'movimentação terra', 'trepadeira',
    'arbustos', 'invasões', 'identificação apagada', 'jumper inadequado', 'quebrados'
]

FORA_DE_DATA = [
    "realizado", "corrigido", "executado"
]

INFORMATIVO = [
    "manutenção realizada", "efetuado", "fazer levantamento", "atividade realizada"
]

NOTA_QUALIFICACAO = None  # será tratado separadamente: quando o valor for numérico puro


from unidecode import unidecode
def fuzzy_contains(text, keywords, cutoff=0.8):
    for word in keywords:
        if unidecode(word.lower()) in unidecode(text.lower()):
            return True
        elif word.lower() in text.lower() :
            return True
        elif word in text:
            return True
        
    for word in text.lower().split():
        if get_close_matches(word, keywords, n=1, cutoff=cutoff):
            return True
    if text.lower() in keywords:
        return True
    
    return False

# 4. Função de classificação
def classify(text):
    # 4.1 Filtro por keyword/fuzzy
    if fuzzy_contains(text, NEGATIVO):

        return "Negativo"

    if fuzzy_contains(text, FORA_DE_DATA):
        return "Fora de Data"

    if fuzzy_contains(text, INFORMATIVO):
        return "Informativo"
    
    if text.isdigit():
        return "Nota de Qualificação"

    if text not in llm_map:
        # 4.2 Prompt para o SmolLM2
        system = {"role": "system", "content": "Você é um classificador de sentimento com rótulos: Positivo, Neutro ou Negativo. Estamos analisando inspeções de torres de linhas de transmissão, leve em conta o que pode atrapalhar ou não uma torre de energia.."}
        user   = {"role": "user",
                "content": f"Classifique este texto em Positivo, Neutro ou Negativo:\n\n\"{text}\""}
        messages = [system, user]
        
        # Aplica template de chat e tokeniza
        input_text = tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
        
        # Gera a resposta breve
        out = model.generate(
            inputs,
            max_new_tokens=20,
            temperature=0.2,
            top_p=0.9,
            do_sample=True,
        )
        reply = tokenizer.decode(out[0], skip_special_tokens=True)

        if 'Positivo'.lower() in reply.lower():
            llm_map[text] = "Positivo"
            return "Positivo"
        elif 'Neutro'.lower() in reply.lower():
            llm_map[text] = "Neutro"
            return "Neutro"
        elif 'Negativo'.lower() in reply.lower():
            llm_map[text] = "Negativo"
            return "Negativo"
    return llm_map[text]



def fuzzy_contains(text, keywords, cutoff=0.8):
    for word in keywords:
        if unidecode(word.lower()) in unidecode(text.lower()):
            return True
        elif word.lower() in text.lower() :
            return True
        elif word in text:
            return True
        
    for word in text.lower().split():
        if get_close_matches(word, keywords, n=1, cutoff=cutoff):
            return True
    if text.lower() in keywords:
        return True
    
    return False

# 4. Função de classificação
def classify2(text):
    # 4.1 Filtro por keyword/fuzzy
    if fuzzy_contains(text, NEGATIVO):
        return "Negativo"

    if fuzzy_contains(text, FORA_DE_DATA):
        return "Fora de Data"

    if fuzzy_contains(text, INFORMATIVO):
        return "Informativo"
    

    if text not in llm_map:
        # 4.2 Prompt para o SmolLM2
        system = {"role": "system", "content": "Você é um classificador de sentimento com rótulos: Positivo, Neutro ou Negativo. Estamos analisando inspeções de torres de linhas de transmissão, leve em conta o que pode atrapalhar ou não uma torre de energia.."}
        user   = {"role": "user",
                "content": f"Classifique este texto em Positivo, Neutro ou Negativo:\n\n\"{text}\""}
        messages = [system, user]
        
        # Aplica template de chat e tokeniza
        input_text = tokenizer.apply_chat_template(messages, tokenize=False)
        inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
        
        # Gera a resposta breve
        out = model.generate(
            inputs,
            max_new_tokens=20,
            temperature=0.2,
            top_p=0.9,
            do_sample=True,
        )
        reply = tokenizer.decode(out[0], skip_special_tokens=True)

        if 'Positivo'.lower() in reply.lower():
            llm_map[text] = "Positivo"
            return "Positivo"
        elif 'Neutro'.lower() in reply.lower():
            llm_map[text] = "Neutro"
            return "Neutro"
        elif 'Negativo'.lower() in reply.lower():
            llm_map[text] = "Negativo"
            return "Negativo"
    return llm_map[text]

In [19]:
texts = df_docs['Texto'].unique()

results = {}
results2 = {}
for idx, t in (enumerate(texts[:])):
    if t not in results:

        results[t] = classify(t)
        results2[t] = classify2(t)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


0 3158
1 3158
2 3158
3 3158
4 3158
5 3158
6 3158
7 3158
8 3158
9 3158
10 3158
11 3158
12 3158
13 3158
14 3158
15 3158
16 3158
17 3158
18 3158
19 3158
20 3158


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


21 3158
22 3158
23 3158
24 3158
25 3158
26 3158
27 3158
28 3158
29 3158
30 3158
31 3158
32 3158
33 3158
34 3158
35 3158
36 3158
37 3158
38 3158
39 3158
40 3158
41 3158
42 3158
43 3158
44 3158
45 3158
46 3158
47 3158
48 3158
49 3158
50 3158
51 3158
52 3158
53 3158
54 3158
55 3158
56 3158
57 3158
58 3158
59 3158
60 3158
61 3158
62 3158
63 3158
64 3158
65 3158
66 3158
67 3158
68 3158
69 3158
70 3158
71 3158
72 3158
73 3158
74 3158
75 3158
76 3158
77 3158
78 3158
79 3158
80 3158
81 3158
82 3158
83 3158
84 3158
85 3158
86 3158
87 3158
88 3158
89 3158
90 3158
91 3158
92 3158
93 3158
94 3158
95 3158
96 3158
97 3158
98 3158
99 3158
100 3158
101 3158
102 3158
103 3158
104 3158
105 3158
106 3158
107 3158
108 3158
109 3158
110 3158
111 3158
112 3158
113 3158
114 3158
115 3158
116 3158
117 3158
118 3158
119 3158
120 3158
121 3158
122 3158
123 3158
124 3158
125 3158
126 3158
127 3158
128 3158
129 3158
130 3158
131 3158
132 3158
133 3158
134 3158
135 3158
136 3158
137 3158
138 3158
139 3158
140 3158

In [20]:
df_docs['Texto_classificado'] = df_docs['Texto'].apply(lambda x: results[x])
df_docs['Texto_classificado_raw'] = df_docs['Texto'].apply(lambda x: results2[x])


len(df_docs[df_docs['Texto_classificado'] == 'Positivo']['Texto'].unique())

699

In [21]:
len(df_docs[df_docs['Texto_classificado'] == 'Negativo']['Texto'].unique())

2373

In [22]:
df_docs[df_docs['Texto_classificado'] == 'Positivo']['Texto'].unique()[:100]

array(['Não Informado', 'No vão', 'Antiga', 'Media. inferior',
       'Fases inferior. média superior', 'Oportunidade', 'Vão total T3',
       'T12', 'Não houve nescessidade de roço', 'Trator T3',
       'Reposição de peças', 'Torre sem anomalia',
       'Dejetos de aves FASE do meio T3', 'Dejetos de pássaro T3',
       'Lado direito externa', 'Fase externa lado esquerdo',
       'Reabrir  faixa lado direito',
       'Reinst. rabicho do cabo PR á estrutura',
       'Esfera de sinalização abrindo.', 'Não ha necessidade de execução',
       'LT ARR-POF lado direito. fase inferior (',
       'LT ARR-BAD lado esquerdo. fase do meio (',
       'Esfera corrida lado direito.', 'Numeração',
       'Em toda a extensão da estrutura', 'Numeração e circuito',
       'Entre o cavalote e o elo bola', 'Prioridade T12.',
       'Prioridade T12', 'Prioridade T12. não oferece risco a opre',
       'Obs: plca de advertência apagada.já fora', 'Inferior',
       'Fase média', 'Antiga no vão', 'Re. oportuni

In [23]:
df_docs.to_csv("./processed_data/medicoes_texto_corrigido.csv", index=False)